In [1]:
import pandas as pd
import numpy as np
import csv
import nltk
import string
import matplotlib.pyplot as plt
# %matplotlib inline
import pip
import seaborn as sns
import random

from pyspark.ml import PipelineModel

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import GridSearchCV
# import sklearn.externals.joblib
import joblib
from nltk.corpus import stopwords

nltk.download('stopwords')
from nltk.tokenize import TweetTokenizer
from nltk.stem.wordnet import WordNetLemmatizer

import re
from collections import Counter
from string import punctuation
# import tweepy
# from tweepy import OAuthHandler
import json
from wordcloud import WordCloud
from pathlib import Path
# import chart_studio
# import chart_studio.plotly as py
# import plotly.graph_objs as go
# from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
# import cufflinks as cf

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
collected_tweets="gs://cloud-project-bucket-3/batman_twitter.csv"
col_names = ['date', 'user_loc', 'followers', 'friends', 'text', 'bbox_coords', \
             'full_name', 'country', 'country_code', 'place_type']
df_batman = pd.read_csv(collected_tweets, names=col_names)

In [3]:
pd.set_option('display.max_colwidth', -1)
plt.style.use('seaborn-white')

/tmp/ipykernel_2389/3856750605.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [4]:
# Sort dataframe by date column
df_batman['date'] = pd.to_datetime(df_batman['date'])
df_batman = df_batman.sort_values(by='date', ascending=True)
df_batman = df_batman.reset_index().drop('index', axis=1)

In [5]:
# helper function to clean tweets
def processTweet(tweet):
    # Remove HTML special entities (e.g. &amp;)
    tweet = re.sub(r'\&\w*;', '', tweet)
    # Convert @username to AT_USER
    tweet = re.sub('@[^\s]+', '', tweet)
    # Remove tickers
    tweet = re.sub(r'\$\w*', '', tweet)
    # To lowercase
    tweet = tweet.lower()
    # Remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*\/\w*', '', tweet)
    # Remove hashtags
    tweet = re.sub(r'#\w*', '', tweet)
    # Remove Punctuation and split 's, 't, 've with a space for filter
    tweet = re.sub(r'[' + punctuation.replace('@', '') + ']+', ' ', tweet)
    # Remove words with 2 or fewer letters
    tweet = re.sub(r'\b\w{1,2}\b', '', tweet)
    # Remove whitespace (including new line characters)
    tweet = re.sub(r'\s\s+', ' ', tweet)
    # Remove single space remaining at the front of the tweet.
    tweet = tweet.lstrip(' ')
    # Remove characters beyond Basic Multilingual Plane (BMP) of Unicode:
    tweet = ''.join(c for c in tweet if c <= '\uFFFF')
    return tweet

In [6]:
# clean dataframe's text column
df_batman['text'] = df_batman['text'].apply(processTweet)

In [7]:
# drop duplicates
df_batman = df_batman.drop_duplicates('text')

In [8]:
# tokenize helper function
def text_process(raw_text):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    # Check characters to see if they are in punctuation
    nopunc = [char for char in list(raw_text) if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)

    # Now just remove any stopwords
    return [word for word in nopunc.lower().split() if word.lower() not in stopwords.words('english')]

In [9]:
def remove_words(word_list):
    remove = ['batman', '...', '“', '”', '’', '…']
    return [w for w in word_list if w not in remove]


In [10]:
# tokenize message column and create a column for tokens
df_batman.to_csv("gs://cloud-project-bucket-3/batman_col_names.csv")
df_batman = df_batman.copy()
df_batman['tokens'] = df_batman['text'].apply(text_process)  # tokenize style 1
df_batman['no_batman'] = df_batman['tokens'].apply(remove_words)  # tokenize style 2
df_batman.to_csv("gs://cloud-project-bucket-3/batman_cleaned.csv")